<a href="https://colab.research.google.com/github/DAISYCHEBET/mlzoomcamp-module-8-2025/blob/main/module8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Module 8 — Deep Learning: Hair Type Classification (ML Zoomcamp Homework)

This notebook contains the full solution for Module 8 of Machine Learning Zoomcamp. The goal is to train a Convolutional Neural Network (CNN) from scratch using PyTorch to classify hair types (curly vs straight).

This notebook includes:

Dataset download

Reproducibility setup

CNN model creation

Training for 10 epochs

Evaluation

In [1]:
# Install optional utilities (torchsummary helps us count model params later)
!pip install torchsummary --quiet

# Download dataset
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip

# Unzip data
!unzip -q data.zip

# Verify dataset structure
!ls -R data

--2025-11-30 08:25:21--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-30T09%3A07%3A37Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-30T08%3A07%3A01Z&ske=2025-11-30T09%3A07%3A37Z&sks=b&skv=2018-11-09&sig=zvZPDJL9ZABr8Y7tdMGU5Drx7civEIzMGQ1XoKfXMJY%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDQ5MjkyMSwibmJmIjoxNzY0NDkxMTIxLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG

In [2]:
!pip install torchsummary --quiet

# Verify PyTorch and CUDA versions
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))

PyTorch version: 2.9.0+cu126
CUDA available: False


In [3]:
import numpy as np
import torch

SEED = 42  # constant seed value

# Set NumPy seed (affects augmentations/data operations)
np.random.seed(SEED)

# Set PyTorch seed for CPU
torch.manual_seed(SEED)

# Set PyTorch seeds for GPU (Colab usually uses CUDA)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# Make cuDNN deterministic
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print("Reproducibility seeds have been set successfully.")
print("CUDA Available:", torch.cuda.is_available())

Reproducibility seeds have been set successfully.
CUDA Available: False


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Required transformations for both train & test
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],   # ImageNet mean
        std=[0.229, 0.224, 0.225]     # ImageNet std
    )
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load datasets from folder structure
# (Make sure you uploaded the data.zip to Colab and extracted it)
train_dataset = datasets.ImageFolder("data/train", transform=train_transforms)
validation_dataset = datasets.ImageFolder("data/test", transform=test_transforms)

# DataLoaders feed data to the model in mini-batches
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(validation_dataset)}")

Training samples: 800
Validation samples: 201


In [5]:
import torch.nn as nn
import torch

class HairTypeCNN(nn.Module):
    def __init__(self):
        super(HairTypeCNN, self).__init__()

        # Convolutional layer: input_channels=3, output_channels=32
        # Kernel size 3x3, stride=1 (default), padding=1 to keep spatial size
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)

        # Activation function ReLU
        self.relu = nn.ReLU()

        # Max pooling layer 2x2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Flatten layer will be applied in forward

        # Fully connected layer: from flattened vector → 64 neurons
        self.fc1 = nn.Linear(32 * 99 * 99, 64)  # 200-3+1=198, then 198/2≈99 after pooling

        # Output layer: 1 neuron for binary classification
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        # Apply convolution
        x = self.conv1(x)
        x = self.relu(x)

        # Apply max pooling
        x = self.pool(x)

        # Flatten tensor
        x = x.view(x.size(0), -1)  # batch_size x features

        # Fully connected layer
        x = self.fc1(x)
        x = self.relu(x)

        # Output layer
        x = self.fc2(x)
        return x  # no sigmoid here; use BCEWithLogitsLoss


# Instantiate model
model = HairTypeCNN()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(model)

HairTypeCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [6]:
import torch.optim as optim
import torch.nn as nn

# Loss function for binary classification
# BCEWithLogitsLoss is appropriate because:
# - The model outputs raw logits
# - Combines sigmoid activation internally
# - Works with a single output neuron
criterion = nn.BCEWithLogitsLoss()

# Optimizer: Stochastic Gradient Descent (SGD) with momentum
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

# ✅ This answers Question 1: The correct loss function is nn.BCEWithLogitsLoss
print("Loss function and optimizer defined.")

Loss function and optimizer defined.


In [7]:
from torchsummary import summary

# Option 1: Using torchsummary (easy, shows per-layer details)
# Input size = (channels, height, width) = (3, 200, 200)
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total trainable parameters: {total_params}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------
Total trainable parameters: 20073473


In [8]:
import torch

num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()  # set model to training mode
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)  # shape: (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # Apply sigmoid to logits for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # Validation step
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)

            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6665, Acc: 0.6112, Val Loss: 0.6511, Val Acc: 0.6617
Epoch 2/10, Loss: 0.5702, Acc: 0.6787, Val Loss: 0.6332, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5207, Acc: 0.7350, Val Loss: 0.6143, Val Acc: 0.6766
Epoch 4/10, Loss: 0.4773, Acc: 0.7600, Val Loss: 0.6049, Val Acc: 0.6617
Epoch 5/10, Loss: 0.4606, Acc: 0.7550, Val Loss: 0.7307, Val Acc: 0.5672
Epoch 6/10, Loss: 0.3954, Acc: 0.8275, Val Loss: 0.6412, Val Acc: 0.6866
Epoch 7/10, Loss: 0.2844, Acc: 0.8838, Val Loss: 0.8307, Val Acc: 0.6816
Epoch 8/10, Loss: 0.2885, Acc: 0.8788, Val Loss: 0.7052, Val Acc: 0.7114
Epoch 9/10, Loss: 0.1882, Acc: 0.9313, Val Loss: 0.9275, Val Acc: 0.6866
Epoch 10/10, Loss: 0.2585, Acc: 0.8912, Val Loss: 0.8158, Val Acc: 0.6915


In [9]:
import numpy as np

# Convert lists to numpy arrays for convenience
train_acc = np.array(history['acc'])
train_loss = np.array(history['loss'])

# Question 3: median of training accuracy
median_acc = np.median(train_acc)
print(f"Median training accuracy: {median_acc:.4f}")

# Question 4: standard deviation of training loss
std_loss = np.std(train_loss)
print(f"Standard deviation of training loss: {std_loss:.4f}")

Median training accuracy: 0.7937
Standard deviation of training loss: 0.1462


In [10]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define augmented training transforms
augmented_train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Recreate the augmented training dataset and loader
train_dataset = datasets.ImageFolder("data/train", transform=augmented_train_transforms)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

# Continue training for 10 more epochs
num_epochs_aug = 10

for epoch in range(num_epochs_aug):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val

    # Save metrics to history
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Aug Epoch {epoch+1}/{num_epochs_aug}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Aug Epoch 1/10, Loss: 0.6307, Acc: 0.6600, Val Loss: 0.5963, Val Acc: 0.7114
Aug Epoch 2/10, Loss: 0.5801, Acc: 0.6750, Val Loss: 0.5968, Val Acc: 0.7214
Aug Epoch 3/10, Loss: 0.5374, Acc: 0.7175, Val Loss: 0.5870, Val Acc: 0.7015
Aug Epoch 4/10, Loss: 0.5502, Acc: 0.7125, Val Loss: 0.5696, Val Acc: 0.7264
Aug Epoch 5/10, Loss: 0.5248, Acc: 0.7412, Val Loss: 0.6706, Val Acc: 0.6766
Aug Epoch 6/10, Loss: 0.5195, Acc: 0.7475, Val Loss: 0.5673, Val Acc: 0.7413
Aug Epoch 7/10, Loss: 0.5158, Acc: 0.7325, Val Loss: 0.5685, Val Acc: 0.6915
Aug Epoch 8/10, Loss: 0.4903, Acc: 0.7488, Val Loss: 0.5941, Val Acc: 0.7015
Aug Epoch 9/10, Loss: 0.4946, Acc: 0.7550, Val Loss: 0.5517, Val Acc: 0.7363
Aug Epoch 10/10, Loss: 0.4662, Acc: 0.7688, Val Loss: 0.5401, Val Acc: 0.7164


In [11]:
import numpy as np

# Extract only the validation portion corresponding to augmentation training
# The last 10 epochs in 'val_loss' and 'val_acc' are from augmented training
aug_val_loss = np.array(history['val_loss'][-10:])
aug_val_acc = np.array(history['val_acc'][-10:])

# Question 5: mean of validation loss for all 10 augmented epochs
mean_val_loss = np.mean(aug_val_loss)
print(f"Mean validation loss (augmented training): {mean_val_loss:.4f}")

# Question 6: average validation accuracy for last 5 epochs
avg_val_acc_last5 = np.mean(aug_val_acc[-5:])
print(f"Average validation accuracy (last 5 epochs): {avg_val_acc_last5:.4f}")

Mean validation loss (augmented training): 0.5842
Average validation accuracy (last 5 epochs): 0.7174
